In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

Load data from csv file

In [2]:
data = pd.read_csv('DR_Demo_Lending_Club.csv', header=0)

In [3]:
#is_bad =data['is_bad']
#features = data.drop('is_bad', axis = 1)

In [4]:
print(data.describe())

                Id        is_bad     annual_inc  debt_to_income  delinq_2yrs  \
count  10000.00000  10000.000000    9999.000000    10000.000000  9995.000000   
mean    5000.50000      0.129500   68203.011539       13.338704     0.148174   
std     2886.89568      0.335769   48590.252757        6.754212     0.506270   
min        1.00000      0.000000    2000.000000        0.000000     0.000000   
25%     2500.75000      0.000000   40000.000000        8.160000     0.000000   
50%     5000.50000      0.000000   58000.000000       13.410000     0.000000   
75%     7500.25000      0.000000   82000.000000       18.692500     0.000000   
max    10000.00000      1.000000  900000.000000       29.990000    11.000000   

       inq_last_6mths  mths_since_last_delinq  mths_since_last_record  \
count     9995.000000             3684.000000              840.000000   
mean         1.066933               35.890337               61.652381   
std          1.476052               22.361443               

In [5]:
data.head(5)

,Id,is_bad,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,Notes,purpose_cat,...,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code
0,1,0,Time Warner Cable,10,MORTGAGE,50000.0,not verified,n,NaN,medical,...,NaN,15.0,0.0,12087,12.1,44.0,f,0.0,1,PC4
1,2,0,Ottawa University,1,RENT,39216.0,not verified,n,Borrower added on 04/14/11 > I will be using...,debt consolidation,...,NaN,4.0,0.0,10114,64.0,5.0,f,0.0,2,PC1
2,3,0,Kennedy Wilson,4,RENT,65000.0,not verified,n,NaN,credit card,...,NaN,4.0,0.0,81,0.6,8.0,f,0.0,3,PC4
3,4,0,TOWN OF PLATTEKILL,10,MORTGAGE,57500.0,not verified,n,NaN,debt consolidation,...,NaN,6.0,0.0,10030,37.1,23.0,f,0.0,2,PC2
4,5,0,Belmont Correctional,10,MORTGAGE,50004.0,VERIFIED - income,n,"I want to consolidate my debt, pay for a vacat...",debt consolidation,...,NaN,8.0,0.0,10740,40.4,21.0,f,0.0,3,PC3


In [6]:
print(list(data.columns))

['Id', 'is_bad', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan', 'Notes', 'purpose_cat', 'purpose', 'zip_code', 'addr_state', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code']


In [7]:
data['home_ownership'].describe()

count     10000
unique        5
top        RENT
freq       4745
Name: home_ownership, dtype: object

In [8]:
data['policy_code'].describe()

count     10000
unique        5
top         PC3
freq       2098
Name: policy_code, dtype: object

In [9]:
data['addr_state'].describe()

count     10000
unique       50
top          CA
freq       1748
Name: addr_state, dtype: object

In [10]:
data['earliest_cr_line'].describe()

count           9995
unique           463
top       11/01/2000
freq              95
Name: earliest_cr_line, dtype: object

In [11]:
data['emp_title'].describe()

count        9408
unique       8214
top       US Army
freq           36
Name: emp_title, dtype: object

We have 8214 unique emp_title. This information does not help to determine rather the load will be good or bad. Most likely the income or other data will reflect rather the borrower has decent income and job. We do not need this information. Just drop it.

In [12]:
data = data.drop('emp_title', axis = 1)

In [13]:
data['initial_list_status'].describe()

count     10000
unique        2
top           f
freq       9983
Name: initial_list_status, dtype: object

In [14]:
data['initial_list_status'].value_counts()

f    9983
m      17
Name: initial_list_status, dtype: int64

This information "Initial_list_status" does not help since 9983 are f and 17 are m. To avoid adding additional noise in our learning model, just drop this information.

In [15]:
data = data.drop('initial_list_status', axis = 1)

In [16]:
print(list(data.columns))

['Id', 'is_bad', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan', 'Notes', 'purpose_cat', 'purpose', 'zip_code', 'addr_state', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code']


In [17]:
data['pymnt_plan'].describe()

count     10000
unique        2
top           n
freq       9998
Name: pymnt_plan, dtype: object

In [18]:
data['pymnt_plan'].value_counts()

n    9998
y       2
Name: pymnt_plan, dtype: int64

This information "pymnt_plan" does not help since 9998 are n and 2 are y. To avoid adding additional noise in our learning model, just drop this information.

In [19]:
data = data.drop('pymnt_plan', axis = 1)

In [20]:
data['policy_code'].describe()

count     10000
unique        5
top         PC3
freq       2098
Name: policy_code, dtype: object

In [24]:
data['purpose'].describe()

count                   9996
unique                  5753
top       Debt Consolidation
freq                     530
Name: purpose, dtype: object

In [26]:
data = data.drop('purpose', axis = 1)

In [27]:
data['purpose_cat'].describe()

count                  10000
unique                    27
top       debt consolidation
freq                    4454
Name: purpose_cat, dtype: object

In [28]:
data['verification_status'].describe()

count            10000
unique               3
top       not verified
freq              4367
Name: verification_status, dtype: object

In [29]:
data['verification_status'].value_counts()

not verified                4367
VERIFIED - income           3214
VERIFIED - income source    2419
Name: verification_status, dtype: int64

In [31]:
print(list(data.columns))

['Id', 'is_bad', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'Notes', 'purpose_cat', 'zip_code', 'addr_state', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code']


In [32]:
data['Notes'].describe()

count     6833
unique    6763
top           
freq        62
Name: Notes, dtype: object

In [33]:
data = data.drop('Notes', axis = 1)

In [34]:
data['annual_inc'].describe()

count      9999.000000
mean      68203.011539
std       48590.252757
min        2000.000000
25%       40000.000000
50%       58000.000000
75%       82000.000000
max      900000.000000
Name: annual_inc, dtype: float64

In [ ]:
data['is_bad'].unique().shape

In [ ]:
data['is_bad'].value_counts()

In [ ]:
data['zip_code'].unique().shape

In [ ]:
data['emp_title'].unique()
data['emp_title'].unique().shape

In [ ]:
data['collections_12_mths_ex_med'].unique()
#data['collections_12_mths_ex_med'].unique().shape

In [ ]:
for i in data:
    print(data[i].value_counts())
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
print(list(data.columns))